<a href="https://colab.research.google.com/github/Siarzis/custom-ai/blob/main/custom_diffusion_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# inherit from the Pytorch "nn.Module"
class custom_DDPM(nn.Module):
    # inputs of the custom_DDPM ->
    # 1. network: the utilized neural network for the diffusion process,
    # 2. n_steps: number of steps in the diffusion process,
    # 3.,4. min_beta, max_beta: the minimum and maximum values for the diffusion parameter
    # 5. device: the device on which the model runs
    # 6. image_chw: shape of the input image.
    def __init__(self, network, n_steps=200, min_beta=10 ** -4, max_beta=0.02, device=None, image_shape=(1, 28, 28)):
        # pass the provided input to the instance of the class
        self.n_steps = n_steps
        self.min_beta = min_beta
        self.max_beta = max_beta
        self.device = device
        self.image_shape = image_shape